In [1]:
import sys
import pathlib
sys.path.append('../../../build')
import numpy as np
import libry as ry
import time

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/quim/stg/jungsu/roboticslab/code/e2'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/quim/stg/jungsu/roboticslab/code/e2'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
# This is the "real" world
RealWorld = ry.Config()
RealWorld.addFile("../../../scenarios/pandasTable.g") 
obj = RealWorld.addFrame("object")
obj.setShape(ry.ST.ssBox, [.1, .1, .1, .02])
obj.setColor([1,0,1])
obj.setPosition([.8,.2,1.])
obj.setMass(0.1)

# This is a Configuration world (where all the computations happen)
C = ry.Config()
C.addFile("../../../scenarios/pandasTable.g") 
V = C.view()
myObj = C.addFrame("myObject")
myObj.setShape(ry.ST.ssBox, [.1, .1, .1, .02])
myObj.setColor([1,0,1])

In [3]:
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
q = C.getJointState()
tau = .01
rate_komo = 10 # run komo once every 10 steps.
horizon_seconds = .5
waypoints = 5 # we could change this adaptatively, based on the discance to the object.
visualize_komo = False
for t in range(1000):
    if t%10 == 0:
        S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    time.sleep(0.01)
    
    
    
    q0 = S.get_q()
    # Cheat Perception! (shortcut)

    if t%rate_komo == 0:
        C.setJointState(q0)
        pos = obj.getPosition()
        quat = obj.getQuaternion()    
        myObj.setPosition(pos)
        myObj.setQuaternion(quat)
        

        # create a komo object
        komo = C.komo_path(1., waypoints, horizon_seconds, True)
        komo.addObjective([1.], 
                        ry.FS.distance, 
                        ["R_finger1_coll", "myObject"], 
                        ry.OT.sos, 
                        [1e2],
                        [0.03])
        ## we are minimizing cost: (d - 0.03) ^ 2
        

        # avoid collision between gripper palm and object
        komo.addObjective([0.,1.], 
                         ry.FS.distance, 
                         ["R_gripper_coll", "myObject"], 
                         ry.OT.ineq, 
                         [1e2])
        # zero velocity at the end
        komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)

        
        komo.optimize()
        komo.getReport()
        if visualize_komo:
            V = komo.view_play(True, 0.2)
        # use the first configuration.
        #print(C.getJointState())
        C.setFrameState(komo.getFrameState(0))
        #print(C.getJointState())
            velq = ( C.getJointState() - q0 ) /  ( horizon_seconds / waypoints )


        S.step(q0 + velq * tau, tau,  ry.ControlMode.position)

** KOMO::run solver:sparse collisions:1 x-dim:70 T:5 k:2 phases:1 stepsPerPhase:5 tau:0.1  #timeSlices:7 #totalDOFs:70 #frames:770
** optimization time:0.104975 (kin:0.003542 coll:0.078649 feat:0.01483 newton: 0.003682) setJointStateCount:162
   sos:60.7347 ineq:0 eq:0.0783223
** KOMO::run solver:sparse collisions:1 x-dim:70 T:5 k:2 phases:1 stepsPerPhase:5 tau:0.1  #timeSlices:7 #totalDOFs:70 #frames:770
** optimization time:0.117368 (kin:0.00109 coll:0.080162 feat:0.019413 newton: 0.008064) setJointStateCount:159
   sos:48.9676 ineq:0 eq:0.103906
** KOMO::run solver:sparse collisions:1 x-dim:70 T:5 k:2 phases:1 stepsPerPhase:5 tau:0.1  #timeSlices:7 #totalDOFs:70 #frames:770
** optimization time:0.1243 (kin:0.000992 coll:0.095766 feat:0.015814 newton: 0.006861) setJointStateCount:181
   sos:66.9333 ineq:0 eq:0.100404
** KOMO::run solver:sparse collisions:1 x-dim:70 T:5 k:2 phases:1 stepsPerPhase:5 tau:0.1  #timeSlices:7 #totalDOFs:70 #frames:770
** optimization time:0.166322 (kin:0.0